In [1]:
import netCDF4 as nc
import numpy as np
import xarray as xr
import warnings
from eofs.xarray import Eof
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import gridspec
import datetime
import seaborn as sns
import cmocean as cmo
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from scipy.stats import linregress
import matplotlib.patches as mpatches
import scipy.io as sio
import matplotlib.colors as mcolors
import matplotlib.patches as patch
import sys
import os
sys.path.append(os.path.abspath('../scripts'))
from proj_utils import *
fig_path      = '../figures'

# Pull Mass Flux Dataset

In [6]:
s3, credentials = init_S3FileSystem(use_earthdata=False, requester_pays=True)
mzz_local_directory = Path('/efs_ecco/mzz-jsons-V4r5/MZZ_mon_mean_native')

In [13]:
mzz_local_file= mzz_local_directory / 'OCEAN_VOLUME_FLUX_mon_mean_native_llc090_ECCOV4r5.json' # Pull monthly velocity 
fs = fsspec.filesystem("reference",     
                       fo=str(mzz_local_file),
                       remote_protocol="s3",
                       remote_options={"anon":False, "requester_pays":True},
                       skip_instance_cache=True)
# 
fs.asynchronous = True
store = zarr.storage.FsspecStore(fs)
ds = xr.open_dataset(store, engine='zarr',
                     consolidated=False, chunks={'time':4, 'Z':50})

In [14]:
bathy_path = user_home_dir + '/efs_ecco/ECCO/V4/r5/netcdf/native/geometry/GRID_GEOMETRY_ECCO_V4r5_native_llc0090.nc'
ds_bathy   = xr.open_dataset(bathy_path).Depth
ds_geom    = xr.open_dataset(bathy_path)

In [15]:
ds_north_vels = proc_dataset(ds, face = 'north', mask = True)
ds_north_geom = proc_dataset(ds_geom, face = 'south', mask = True)

In [ ]:
ds_south_vels = 

In [ ]:
def proc_dataset(ds, face = 'north', mask = True):
    ds = get_na_tile(ds)
    ds = subset_tgb_box(ds)
    if face == 'north':
        ds = ds.sel(i = slice(ds.i[0]), i_g = slice(ds.i_g[0])).squeeze()
        if mask == 'True':
            mask_da = xr.open_dataarray('../data/north_face_mask.nc')
            ds = ds.where(mask_da)
    return(ds)